In [1]:
#  https://www.kaggle.com/code/werooring/ch11-eda

In [ ]:
# !unzip '/content/drive/MyDrive/Colab Notebooks/aerial-cactus-identification.zip' -d '/content/drive/MyDrive/Colab Notebooks/data/'

In [ ]:
!unzip '/content/drive/MyDrive/Colab Notebooks/aerial-cactus-identification/test.zip'

In [ ]:
!unzip '/content/drive/MyDrive/Colab Notebooks/aerial-cactus-identification/train.zip'

In [8]:
import pandas as pd

# 데이터 경로
data_path = '/content/drive/MyDrive/Colab Notebooks/aerial-cactus-identification/'
labels = pd.read_csv(data_path + 'train.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [3]:
import torch # 파이토치 
import random
import numpy as np
import os

# 시드값 고정
seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)                # 파이썬 난수 생성기 시드 고정
np.random.seed(seed)             # 넘파이 난수 생성기 시드 고정
torch.manual_seed(seed)          # 파이토치 난수 생성기 시드 고정 (CPU 사용 시)
torch.cuda.manual_seed(seed)     # 파이토치 난수 생성기 시드 고정 (GPU 사용 시)
torch.cuda.manual_seed_all(seed) # 파이토치 난수 생성기 시드 고정 (멀티GPU 사용 시)
torch.backends.cudnn.deterministic = True # 확정적 연산 사용
torch.backends.cudnn.benchmark = False    # 벤치마크 기능 해제
torch.backends.cudnn.enabled = False      # cudnn 사용 해제

In [4]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [5]:
device

device(type='cuda')

In [6]:
# 1 훈련/검증 데이터 분리
# 2. 데이터셋 클래스 정의
# 3. 이미지 변환기 정의  --> 개선사항
# 4. 데이터셋 생성
# 5. 데이터 로드 생성

In [9]:
from sklearn.model_selection import train_test_split

# 훈련 데이터, 검증 데이터 분리
train, valid = train_test_split(labels, 
                                test_size=0.1,
                                stratify=labels['has_cactus'],
                                random_state=50)

In [10]:
print('훈련 데이터 개수:', len(train))
print('검증 데이터 개수:', len(valid))

훈련 데이터 개수: 15750
검증 데이터 개수: 1750


In [11]:
import cv2 # OpenCV 라이브러리
from torch.utils.data import Dataset # 데이터 생성을 위한 클래스

class ImageDataset(Dataset):
    # 초기화 메서드(생성자)
    def __init__(self, df, img_dir='./', transform=None):
        super().__init__() # 상속받은 Dataset의 생성자 호출
        # 전달받은 인수들 저장
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
    
    # 데이터셋 크기 반환 메서드 
    def __len__(self):
        return len(self.df)
    
    # 인덱스(idx)에 해당하는 데이터 반환 메서드 
    def __getitem__(self, idx):
        img_id = self.df.iloc[idx, 0]    # 이미지 ID
        img_path = self.img_dir + img_id # 이미지 파일 경로 
        image = cv2.imread(img_path)     # 이미지 파일 읽기 
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # 이미지 색상 보정
        label = self.df.iloc[idx, 1]     # 이미지 레이블(타깃값)

        if self.transform is not None:
            image = self.transform(image) # 변환기가 있다면 이미지 변환
        return image, label

In [14]:
# 3. 이미지 변환기 정의  --> 개선사항
from torchvision import transforms # 이미지 변환을 위한 모듈
# 훈련용 데이터 변환기
trainform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Pad(32,padding_mode='symmetric'),  #32 x 32 이미지 주변에 32두께의 패딩을 두른다, 원본데이터를  상하 좌우 대칭
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))  # 이미지넷의 데이터의 RGB 평균과 표준편차
])

# 검증 및 테스트 데이터용 변환기
trainform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Pad(32,padding_mode='symmetric'),  #32 x 32 이미지 주변에 32두께의 패딩을 두른다, 원본데이터를  상하 좌우 대칭    
    transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))  # 이미지넷의 데이터의 RGB 평균과 표준편차
])

In [15]:
dataset_train = ImageDataset(df=train, img_dir='train/', transform=trainform_train)
dataset_valid = ImageDataset(df=valid, img_dir='train/', transform=trainform_test)

In [16]:
from torch.utils.data import DataLoader # 데이터 로더 클래스

loader_train = DataLoader(dataset=dataset_train, batch_size=32, shuffle=True)
loader_valid = DataLoader(dataset=dataset_valid, batch_size=32, shuffle=False)

In [18]:
data,label = next(iter(loader_train))
data.shape, label.shape

(torch.Size([32, 3, 96, 96]), torch.Size([32]))

In [17]:
# 성능 개선용 CNN
# [합성곱,배치정규화,최대폴링] x 5 , 평균폴링 ,평탄화, FC, FC, output

In [28]:
import torch.nn as nn # 신경망 모듈
import torch.nn.functional as F # 신경망 모듈에서 자주 사용되는 함수
class Model(nn.Module):
  # 신경망 계층 정의 
  def __init__(self):
    super().__init__() # 상속받은 nn.Module의 __init__() 메서드 호출
    # 32, 3, 96, 96
    self.layer1 = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=32,kernel_size=3, padding=2),
        nn.BatchNorm2d(32),
        nn.MaxPool2d(kernel_size=2)
    )
    # 32,32,49,49
    self.layer2 = nn.Sequential(
        nn.Conv2d(in_channels=32, out_channels=64,kernel_size=3, padding=2),
        nn.BatchNorm2d(64),
        nn.MaxPool2d(kernel_size=2)
    )
    # 32,64,25,25
    self.layer3 = nn.Sequential(
        nn.Conv2d(in_channels=64, out_channels=128,kernel_size=3, padding=2),
        nn.BatchNorm2d(128),
        nn.MaxPool2d(kernel_size=2)
    )
    # 32,128,13,13
    self.layer4 = nn.Sequential(
        nn.Conv2d(in_channels=128, out_channels=256,kernel_size=3, padding=2),
        nn.BatchNorm2d(256),
        nn.MaxPool2d(kernel_size=2)
    )
    # 32,256,7,7
    self.layer5 = nn.Sequential(
        nn.Conv2d(in_channels=256, out_channels=512,kernel_size=3, padding=2),
        nn.BatchNorm2d(512),
        nn.MaxPool2d(kernel_size=2)
    )
    # 32,512,4,4
    # 평균 풀링 계층 
    self.avg_pool = nn.AvgPool2d(kernel_size=2) 
    # 32,512,2,2

    # 평탄화 
    self.fc1 = nn.Linear(in_features=512*2*2, out_features= 64)
    self.fc2 = nn.Linear(in_features=64, out_features= 2)
  # 순전파 출력 정의
  def forward(self, x):
      x = self.layer1(x)
      x = self.layer2(x)
      x = self.layer3(x)
      x = self.layer4(x)
      x = self.layer5(x)
      x = self.avg_pool(x)
      x = x.view(-1, 512 * 2 * 2) # 평탄화
      x = self.fc1(x)
      x = self.fc2(x)
      return x    

In [29]:
model = Model().to(device)
# 손실함수
criterion = nn.CrossEntropyLoss()
# 옵티마이저
optimizer = torch.optim.Adamax(model.parameters(), lr=0.00006)

In [ ]:
epochs = 70 # 총 에폭
# 총 에폭만큼 반복
for epoch in range(epochs):
    epoch_loss = 0 # 에폭별 손실값 초기화
    
    # '반복 횟수'만큼 반복 
    for images, labels in loader_train:
        # 이미지, 레이블 데이터 미니배치를 장비에 할당 
        images = images.to(device)
        labels = labels.to(device)
        
        # 옵티마이저 내 기울기 초기화
        optimizer.zero_grad()
        # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        # 손실 함수를 활용해 outputs와 labels의 손실값 계산
        loss = criterion(outputs, labels)
        # 현재 배치에서의 손실 추가
        epoch_loss += loss.item() 
        # 역전파 수행
        loss.backward()
        # 가중치 갱신
        optimizer.step()
        
    # 훈련 데이터 손실값 출력
    print(f'에폭 [{epoch+1}/{epochs}] - 손실값: {epoch_loss/len(loader_train):.4f}')